In [1]:
import numpy as np
EPSILON=0.000001

In [2]:
def myround(num):
    num = num if abs(num)>EPSILON else 0
    return num

In [3]:
def case_check(pon, poffs, c, con):
    if poffs>1/2 * pon + 2*c and pon < 4*c and poffs < 1-2*c and poffs < pon + con:
        return 1
    else:
        return 0

In [4]:
def calculate_prior_demand(pon, poffs, c,con):
    alpha_o = 1/(2*con) *(2-2*pon-4*c)*(2*c-1/2*pon)
    alpha_so_prior = 1/(2*con) *(2-6*c - poffs-1/2 *pon)*(poffs-1/2*pon-2*c)
    alpha_ss_prior  = 1/(con) *(con-poffs+pon)*( 1- poffs -2*c)
    alpha_s = alpha_so_prior + alpha_ss_prior
    alpha_l = 1-alpha_o-alpha_s
    
    alpha_o = myround(alpha_o)
    alpha_s = myround(alpha_s)
    alpha_l = myround(alpha_l)
    
    return alpha_o, alpha_s, alpha_l

In [5]:
def calculate_store_demand(pon, poff, c, con, alpha_s):
    if pon<=poff<=pon+con:
        prop_ss = 1/con *(con - poff+pon)*(1-poff)
        prop_so= 1/(2*con) *(2-poff-pon)*(poff-pon)
    elif poff > pon+con:
        prop_ss=0
        prop_so=1/(2*con) *(2-2*pon-con)*con
    else:
        prop_ss=1/(con)*(1-poff)*con
        prop_so=0
        
    prop_ss = prop_ss
    prop_so = prop_so
    
    alpha_ss = myround(alpha_s * prop_ss)
    alpha_so = myround(alpha_s * prop_so)
    
    return alpha_ss, alpha_so

In [6]:
def cal_profit(pon, poff,cr,alpha_o,alpha_ss,alpha_so):
    store_profit = 1/2 *alpha_ss * poff + 1/2 * alpha_so * pon      #  w.p 1/2, we have a=a_H,
    online_direct_profit = alpha_o*( 1/2 * pon + 1/4 * (pon - cr))  # w.p 1/2, we have a=a_H, if a=a_L, w.p. 1/2, product return
    profit= 1/2 * (online_direct_profit  +  store_profit)         # w.p 1/2, we have b=b_H
    return profit

In [7]:
def cal_profit_store(pon, poff,alpha_ss,alpha_so):
    """
    This method optimizes the retailer's profit from instore consumers
    """
    store_profit = 1/2 *alpha_ss * poff + 1/2 * alpha_so * pon      #  w.p 1/2, we have a=a_H,
    return 1/2*store_profit

In [22]:
def FindRationalExpectations(pon, poffs, c, con, alpha_s):
    max_store_profit = 0
    max_store_price = 0
    for poff in np.arange(0, 1, 0.01):
        alpha_ss, alpha_so = calculate_store_demand(pon=pon, poff=poff,
                                                    c=c, con=con, alpha_s=alpha_s)
        if alpha_ss > 1 or alpha_ss < 0.0 or alpha_so > 1 or alpha_so < 0.0:
            print("error instore demand")
            print("poffs: {}, poff: {},ss: {},so: {}".format(
                poffs, poff, alpha_ss, alpha_so))
        current_profit_store = cal_profit_store(pon=pon, poff=poff, alpha_ss=alpha_ss, alpha_so=alpha_so)
        if max_store_profit < current_profit_store:
            max_store_profit = current_profit_store

    if abs(poffs - max_store_price) >= EPSILON:
        return False, None, None  # no RE in current poffs
    else:
        return True, max_store_profit, max_store_price


In [23]:
# def solve_equilibrium(c, cr, con):
#     optimal_total_profit = 0
#     ponstar = 0
#     for pon in np.arange(0, 1, 0.01):
# #         print("-------------current pon: {}---------------".format(pon))
#         store_profit_givenpon = 0
#         alpha_ss_givenpon =0
#         alpha_so_givenpon =0
#         alpha_o_givenpon=0
#         total_profit_givenpon=0
        
#         optimal_RE_profit_givenpon=0 # the profit from a potential RE, given pon
# #         current_total_profit=0 # RE that gives the highest profit, given a pon
        
#         for poffs in np.arange(0, 1, 0.01):
#             current_case = case_check(pon=pon, poffs=poffs, c=c, con=con)
#             if current_case:  # print("current poffs: {}".format(poffs))
#                 alpha_o, alpha_s, alpha_l = calculate_prior_demand(pon=pon, poffs=poffs, c=c, con=con)
#                 if alpha_o>1 or alpha_o<0 or alpha_s>1 or alpha_s<0: 
#                     print("error prior demand")
#             else:
#                 continue
            
#             # if prior offline demand is zero, find the next poffs
#             if not alpha_s:
#                 alpha_ss_givenpon =0
#                 alpha_so_givenpon =0
#                 alpha_o_givenpon=alpha_o
#                 total_profit_givenpon_givenpoffs=cal_profit(pon=pon, poff=poffs,cr=cr,alpha_o=alpha_o_givenpon,
#                                         alpha_ss=alpha_ss_givenpon,alpha_so=alpha_so_givenpon)
#                 if total_profit_givenpon<total_profit_givenpon_givenpoffs:
#                     total_profit_givenpon=total_profit_givenpon
#                 continue
            
#             # start to solve for rational expectation equilibrium    
#             poffstar = 0    
#             potential_alpha_ss =0
#             potential_alpha_so =0
#             potential_alpha_o=0
#             potential_store_profit = 0
            
#             for poff in np.arange(0, 1.01, 0.01):
#                 alpha_ss, alpha_so = calculate_store_demand(pon=pon,poff=poff, 
#                                                             c=c, con=con,alpha_s=alpha_s)
#                 if alpha_ss>1 or alpha_ss<0.0 or alpha_so>1 or alpha_so<0.0: 
#                     print("error instore demand")
#                     print("poffs: {}, poff: {},ss: {},so: {}".format(
#                         poffs, poff, prop_ss,prop_so))
                        
#                 # solve optimal poff w.r.t. total profit 
#                 # current_profit = cal_profit(pon=pon, poff=poff,cr=cr,alpha_o=alpha_o,alpha_ss=alpha_ss,alpha_so=alpha_so) 
#                 # solve optimal poff w.r.t. store profit. Given pon and poffs, the two methods are equivalent.
#                 current_profit_store = cal_profit_store(pon=pon, poff=poff, alpha_ss=alpha_ss, alpha_so=alpha_so) 
#                 if potential_store_profit < current_profit_store:
#                     potential_store_profit = current_profit_store
#                     poffstar=poff
#                     potential_alpha_ss=alpha_ss
#                     potential_alpha_so=alpha_so
#                     potential_alpha_o=alpha_o
#             # print("optimal profit:{},poffs: {},poffstar: {}".format(optimal_profit,poffs,poffstar))
            
#             # If poffs= poff*, we find a possible RE 
#             if abs(poffs - poffstar)>=EPSILON:
#                 continue
#             else:
#             # print("alpha so:", optimal_alpha_so)
#             # Given pon and poffs, find a potential RE.
#                 RE_profit_givenpon_givenpoffs=cal_profit(pon=pon, poff=poffstar,cr=cr,alpha_o=potential_alpha_o,
#                                         alpha_ss=potential_alpha_ss,alpha_so=potential_alpha_so) 
#                 if RE_profit_givenpon <  RE_profit_givenpon_givenpoffs:
#                     RE_profit_givenpon=RE_profit_givenpon_givenpoffs
                    
#                     optimal_alpha_ss=potential_alpha_ss
#                     optimal_alpha_so=potential_alpha_so
#                     optimal_alpha_o=potential_alpha_o
#                     poffstar=poff
#                     print("current pon: {}, poffs:{}".format(pon, poffs))
#                     print("poffstar: {}; optimal profit:{}, store profit:{}, online profit:{}".format(
#                         poffstar, current_optimal_RE_profit, optimal_store_profit,current_optimal_RE_profit- optimal_store_profit))
#                 # print("{}".format(optimal_alpha_o, optimal_alpha_ss, optimal_alpha_so))

#         if optimal_total_profit<current_optimal_RE_profit:
#             optimal_total_profit = current_optimal_RE_profit
#             ponstar=pon
#     if ponstar:
#         print("ponstar: {}, optimal profit: {}".format(ponstar, optimal_total_profit))

In [24]:
# -*- coding: utf-8 -*-
"""
@Created at 2022/4/29 10:19
@Author: Kurt
@file:main.py
@Desc:
"""
def solve_equilibrium(c, cr, con):
    optimal_total_profit = 0  # the RE that maximizes total profit for any pon.
    optimal_poffs = 0
    optimal_poff = 0
    optimal_pon = 0
    for pon in np.arange(0, 1, 0.01):
        # given pon, find potential REs.
        RE_profit_givenpon = 0  # the RE that maximizes total profit given pon.
        poffs_givenpon = 0
        poffstar_givenpon = 0

        # star to find REs
        for poffs in np.arange(0, 1, 0.01):
            current_case = case_check(pon=pon, poffs=poffs, c=c, con=con)
            if current_case:  # print("current poffs: {}".format(poffs))
                alpha_o, alpha_s, alpha_l = calculate_prior_demand(pon=pon, poffs=poffs, c=c, con=con)
                if alpha_o > 1 or alpha_o < 0 or alpha_s > 1 or alpha_s < 0:
                    print("error prior demand")
            else:
                continue

            if not alpha_s:
                # for a zero prior store demand, RE exists
                RE_profit_givenpon_zero_store_demand = cal_profit(pon=pon, cr=cr, alpha_o=alpha_o, store_profit=0)
                if RE_profit_givenpon < RE_profit_givenpon_zero_store_demand:
                    RE_profit_givenpon = RE_profit_givenpon_zero_store_demand
                continue  # look for the next poffs

            # if prior store demand >0, start to find a RE
            found, store_profit, store_price = FindRationalExpectations(pon=pon, poffs=poffs, c=c, con=con,
                                                                        alpha_s=alpha_s)

            if found:
                # Given pon, if we find a RE in the current poffs, compare it with optimal RE collected in other poffs.
                potential_RE_profit_givenpon = cal_profit(pon=pon, cr=cr, alpha_o=alpha_o,
                                                          store_profit=store_profit)
                if RE_profit_givenpon < potential_RE_profit_givenpon:
                    RE_profit_givenpon = potential_RE_profit_givenpon
                    poffs_givenpon = store_price
                    poffstar_givenpon = store_price
            else:
                continue

        if optimal_total_profit < RE_profit_givenpon:
            optimal_total_profit = RE_profit_givenpon
            optimal_poff = poffstar_givenpon
            optimal_poffs = poffs_givenpon
            optimal_pon = pon
            
    if optimal_pon:
        print("ponstar: {}, poffs:{}, poffstar:{}, profit:{}".format(
            optimal_pon, optimal_poffs, optimal_poff, optimal_total_profit))

    return optimal_pon, optimal_poffs, optimal_poff, optimal_total_profit

In [25]:
for c in np.arange(0.0, 0.2,0.05):
    print("----current c: {}-------------".format(c))
    solve_equilibrium(c=c, cr=0.1, con=0.2)

----current c: 0.0-------------
----current c: 0.05-------------
----current c: 0.1-------------
----current c: 0.15000000000000002-------------


In [32]:
con=0.2
cr=0.1
c=0.05
pon=0.19
poff=0.27
poffs=0.27

alpha_o, alpha_s, alpha_l = calculate_prior_demand(pon=pon, poffs=poffs, c=c, con=con)
print("alpha s:", alpha_s)

alpha_ss, alpha_so = calculate_store_demand(pon=pon,poff=poff, c=c, con=con,alpha_s=alpha_s)

print(alpha_o, alpha_ss, alpha_so)

alpha_o = 1/(2*con) *(2-2*pon-4*c)*(2*c-1/2*pon)
profit_online = 1/2 *alpha_o*( 1/2 * pon + 1/4 * (pon - cr))
store=cal_profit_store(pon=pon, poff=poff, alpha_ss=alpha_ss, alpha_so=alpha_so)
total=cal_profit(pon=pon, poff=poffs,cr=cr,alpha_o=alpha_o,alpha_ss=alpha_ss,alpha_so=alpha_so)
print("store profit: {:.5}, totoal profit: {:.5}".format(store,total))
print("online profit: {:.5}, {:.5}".format(total-store, profit_online))

alpha s: 0.6283125
0.017750000000000016 0.275200875 0.19352025000000006
store profit: 0.027768, totoal profit: 0.028811
online profit: 0.0010428, 0.0010428


In [35]:
def cal_store_profit(pon, poff, alpha_ss, alpha_so):
    """
    This method optimizes the retailer's profit from instore consumers
    """
    store_profit = 1 / 2 * alpha_ss * poff + 1 / 2 * alpha_so * pon  # w.p 1/2, we have a=a_H,
    return 1 / 2 * store_profit

In [41]:
def FindRationalExpectations(pon, poffs, c, con, alpha_s):
    max_store_profit = 0
    max_store_price = 0
    for poff in np.arange(0, 1, 0.01):
        alpha_ss, alpha_so = calculate_store_demand(pon=pon, poff=poff,
                                                    c=c, con=con, alpha_s=alpha_s)
        if alpha_ss > 1 or alpha_ss < 0.0 or alpha_so > 1 or alpha_so < 0.0:
            print("error instore demand")
            print("poffs: {}, poff: {},ss: {},so: {}".format(
                poffs, poff, alpha_ss, alpha_so))
        current_profit_store = cal_store_profit(pon=pon, poff=poff, alpha_ss=alpha_ss, alpha_so=alpha_so)
        print("current poff: {:.5}, current store profit: {:.5}".format(poff, current_profit_store))
        if max_store_profit < current_profit_store:
            max_store_profit = current_profit_store
            max_store_price = poff

    print("poffs:{}, Max store pirce：{}".format(poffs, max_store_price))

    if abs(poffs - max_store_price) >= EPSILON:
        return False, None, None  # no RE in current poffs
    else:
        return True, max_store_profit, max_store_price


In [42]:
FindRationalExpectations(pon, poffs, c, con, alpha_s)

current poff: 0.0, current store profit: 0.0
current poff: 0.01, current store profit: 0.0015551
current poff: 0.02, current store profit: 0.0030787
current poff: 0.03, current store profit: 0.004571
current poff: 0.04, current store profit: 0.0060318
current poff: 0.05, current store profit: 0.0074612
current poff: 0.06, current store profit: 0.0088592
current poff: 0.07, current store profit: 0.010226
current poff: 0.08, current store profit: 0.011561
current poff: 0.09, current store profit: 0.012865
current poff: 0.1, current store profit: 0.014137
current poff: 0.11, current store profit: 0.015378
current poff: 0.12, current store profit: 0.016587
current poff: 0.13, current store profit: 0.017766
current poff: 0.14, current store profit: 0.018912
current poff: 0.15, current store profit: 0.020027
current poff: 0.16, current store profit: 0.021111
current poff: 0.17, current store profit: 0.022164
current poff: 0.18, current store profit: 0.023185
current poff: 0.19, current store

(True, 0.027768270937500006, 0.27)

In [2]:
pon=0.43
poffs=0.7
c=0.24
con=0.1
scenario=6

In [12]:
2 - min(1, 1 / 2 * pon + 4 * c, 3 / 2 * pon) - min(1, 1 / 2 * pon + 4 * c)

0.355

In [21]:
if 1 - pon - 2 * c >= 2 * c - 1 / 2 * pon:
    print("1")
    if con >= 1 - pon - 2 * c:
        alpha_o = 1 / (2 * con) * (2 - 2 * pon - 4 * c) * (2 * c - 1 / 2 * pon)
        alpha_so_prior = 1 / (2 * con) * (1 - 1 / 2 * pon - 4 * c) * (1 - 1 / 2 * pon - 4 * c)
    elif con >= 2 * c - 1 / 2 * pon:
        alpha_o = 1 / (2 * con) * (2 - 2 * pon - 4 * c) * (2 * c - 1 / 2 * pon)
        alpha_so_prior = 1 / (2 * con) * (2 - 3 / 2 * pon - 6 * c - con) * (con - 2 * c + 1 / 2 * pon)
    else:
        alpha_o = 1 / (2 * con) * con * (2 - 3 * pon - 2 * con)
        alpha_so_prior = 0
else:
    # assume that 1-pon-2c < 2 *c -1/2 * pon. SO will not emerge.
    print("2")
    alpha_so_prior = 0
    if pon >= 2 / 3:
        print("2.1")
        alpha_o = 0
    else:
        print("2.2")
        if con > 1 / 2 * (1 - 3 / 2 * pon):
            print("2.2.1")
            alpha_o = 1 / (2 * con) * (1 - 3 / 2 * pon) * 1 / 2 * (1 - 3 / 2 * pon)
        else:
            print("2.2.2")
            alpha_o = 1 / (2 * con) * con * (2 - 2 * con - 3 * pon)
print(alpha_o, alpha_so_prior)

2
2.2
2.2.2
0.255 0


In [22]:
pon =0.35
poffs =0.25
c=0.24
con=0.1
1 / 2 * pon + 2 * c >= poffs and poffs < 1 - 2 * c and 1 / 2 * poffs - 3 / 4 * pon + c >= con

True

In [25]:
 1 / 2 * poffs - 3 / 4 * pon + c >= con

True